In [1]:
from db_utils import retrieve_data
import os
import pandas as pd
import sys
sys.path.insert(0, '../')
from features import build_features
from model.train import train_and_pkl
from sklearn.model_selection import TimeSeriesSplit
path = r"c:\Users\brtnl\OneDrive\Desktop\code\sports_betting"
os.chdir(path)

pd.set_option('display.max_columns', None)

In [2]:
# The final will be merged venue and game df.
venue_df = retrieve_data("cfb", "venues")
game_df = retrieve_data("cfb", "games")
df = pd.merge(game_df, venue_df, "left", left_on=["venue_id", "venue"], right_on=["id", "name"], suffixes=("", "_venue"))
fp = build_features.FeaturePipeline(df)
df = fp.run()

In [10]:
first_of_each_year = df.groupby('season').head(1).index
first_of_each_year

Index([0, 1068, 1283, 1312, 1460, 7483, 8470, 10314, 11098, 12236, 17504,
       21212],
      dtype='int64')

In [4]:
df.reset_index(drop=True, inplace=True)

In [5]:
df

season  week  neutral_site  conference_game  attendance  \
0        2013     2             0                1     37533.0   
1        2013     2             0                0     15823.0   
2        2013     2             0                1     32465.0   
3        2013     2             0                0     41170.0   
4        2013     2             0                0     70401.0   
...       ...   ...           ...              ...         ...   
25057    2024     1             1                0         0.0   
25058    2024     1             1                0         0.0   
25059    2024     1             1                0         0.0   
25060    2024     1             1                0         0.0   
25061    2024     1             1                0         0.0   

       home_pregame_elo  away_pregame_elo  excitement_index   latitude  \
0                1450.0            1241.0          0.000000  35.596849   
1                1291.0            1681.0          0.000000  25.752501   
2                1383.0            1295.0          0.000000  42.335104   
3                   0.0               0.0          0.000000  32.709660   
4                1645.0            1431.0          0.000000  42.728147   
...                 ...               ...               ...        ...   
25057               0.0               0.0          0.000000  29.752199   
25058               0.0               0.0          4.820656  33.154444   
25059            2063.0            2173.0          8.450324  25.957967   
25060            2056.0            2182.0          6.155203  32.747778   
25061            2169.0            2196.0          2.819260  33.755578   

       longitude  capacity   elevation  constructionyear  \
0     -77.364564   50000.0   19.056454            1963.0   
1     -80.377891   20000.0    1.718582            1995.0   
2     -71.166441   44500.0   39.826344            1957.0   
3     -97.368084   46000.0  185.438705            1929.0   
4     -84.484853   75005.0  254.879486            1923.0   
...          ...       ...         ...               ...   
25057 -95.352415   22000.0   13.681284            2012.0   
25058 -96.835278   20500.0  181.381592            2005.0   
25059 -80.238860   64767.0    2.624486            1987.0   
25060 -97.092778  100000.0  172.504730            2009.0   
25061 -84.400881   75000.0  307.900000            2017.0   

       home_days_since_last_game  away_days_since_last_game  \
0                            5.0                        6.0   
1                            6.0                        8.0   
2                            6.0                        8.0   
3                            7.0                        9.0   
4                            8.0                        7.0   
...                          ...                        ...   
25057                       15.0                       15.0   
25058                       16.0                       16.0   
25059                        9.0                        7.0   
25060                        9.0                        9.0   
25061                       11.0                       10.0   

       season_type_postseason  season_type_regular  \
0                           0                    1   
1                           0                    1   
2                           0                    1   
3                           0                    1   
4                           0                    1   
...                       ...                  ...   
25057                       1                    0   
25058                       1                    0   
25059                       1                    0   
25060                       1                    0   
25061                       1                    0   

       season_type_spring_postseason  season_type_spring_regular  \
0                                  0                           0   
1                                  0                          

In [6]:
col = "home_h1"
df.reset_index(drop=True, inplace=True)
X = df.drop(columns=[col])
y = df[col]

In [7]:
train_and_pkl(X, y)

               Feature  Percentage
7     excitement_index   29.049730
1636           away_h1   20.022952
1634           home_h2   12.376845
1637           away_h2    5.508426
0               season    3.476607
Model Performance (Booster):
R2 Score:
 0.34363179219744966
MSE:
 79.98146693738832
MAE:
 6.92876213961525


In [8]:
# tscv = TimeSeriesSplit(5)
# for fold, (train_index, test_index) in enumerate(tscv.split(X)):
#     print("Fold: {}".format(fold))
#     print("TRAIN indices:", train_index, "\n", "TEST indices:", test_index)
#     print("\n")
#     X_train, X_test = X.iloc[train_index], X.iloc[test_index]
#     y_train, y_test = y.iloc[train_index], y.iloc[test_index]